# System Information

Autogenerated info about each system

In [1]:
# Plotting setup
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import ticker
from IPython.display import display, display_markdown

import pandas as pd
import os, sys
from pprint import pprint
sys.path.extend(('../../reframe', '../../')) # the `modules` package imports `reframe` so need that in sys.modules
import modules

import json, pprint
from collections import defaultdict

pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

In [2]:
 # load all sysinfo.json files:
sysinfos = {} # key-> reframe "system:partition", value-> nested dict of values
for path in modules.utils.find_run_outputs(root='../../output', test='Sysinfo', ext='.json'):
    # load metadata:
    meta = modules.utils.parse_path_metadata(path)
    syspart = '%s:%s' % (meta['sysname'], meta['partition']) # throw away environment
    with open(path) as f:
        sysinfos[syspart] = json.load(f)
if not sysinfos:
    print('No data')

# System-level Information

In [3]:
# Derive and tabulate calculated values:
general_table = []
general_cols = ['Number of nodes', 'Total CPUs', 'Total memory']
for syspart, sysinfo in sysinfos.items():
    
    num_nodes = len([hostdata['hostname'] for hostdata in sysinfo.values()])
    
    total_cpus = sum(int(modules.utils.get_nested(hostdata, 'cpu.CPU(s)')) for hostdata in sysinfo.values())
    
    mems = [modules.utils.get_nested(hostdata, 'memory.total') for hostdata in sysinfo.values()]
    mem_units = list(set(modules.utils.split_numeric(m)[1] for m in mems))
    if len(mem_units) > 1:
        raise NotImplementedError('Cannot cope with different units for memory size across cluster: %r' % mems)
    total_mem = '%s %s' % (sum(int(modules.utils.split_numeric(m)[0]) for m in mems), mem_units[0])
    
    general_table.append([num_nodes, total_cpus, total_mem])
    general_df = pd.DataFrame(general_table, index=list(sysinfos.keys()), columns=general_cols, dtype=str)
    display(general_df)

,Number of nodes,Total CPUs,Total memory
csd3:cclake-ib-icc19-impi19-ucx,56,3136,10976 G
alaska:ib-gcc9-openmpi4-ucx,56,3136,10976 G


,Number of nodes,Total CPUs,Total memory
csd3:cclake-ib-icc19-impi19-ucx,56,3136,10976 G
alaska:ib-gcc9-openmpi4-ucx,16,1024,2096 G


# Node Details

In [4]:
# Define general tables:
TABLES = {'Operating system': {'OS':'os.release.PRETTY_NAME', 'Kernel':'os.kernel'},
          'Chassis': {'model':'chassis.product_name', 'vendor':'chassis.sys_vendor'},
          'CPU': {'architecture':'cpu.Architecture', 'model':'cpu.Model name',
                  'cpus /node':'cpu.CPU(s)', 'sockets /node':'cpu.Socket(s)', 'cores /socket':'cpu.Core(s) per socket',
                  'threads /core':'cpu.Thread(s) per core',
                 },
          'Memory': {'memory /node':'memory.total', 'type':'memory.types'},
         }
    
# Group system info across reframe partitions and tabulate:
for table_title, table_contents in TABLES.items():
    table = {}
    for syspart, sysinfo in sysinfos.items():
        table[syspart] = {}
        for table_label, datakey in table_contents.items():
            for hostname, nodedata in sysinfo.items():
                val = modules.utils.get_nested(nodedata, datakey)
                table[syspart].setdefault(table_label, set()).add(val)
                
    display_markdown('### %s' % table_title, raw=True)
    df = pd.DataFrame(table)
    df = df.applymap(modules.utils.singleval)
    display(df.transpose())

### Operating system

,OS,Kernel
csd3:cclake-ib-icc19-impi19-ucx,CentOS Linux 7 (Core),3.10.0-1127.19.1.el7.x86_64
alaska:ib-gcc9-openmpi4-ucx,CentOS Linux 7 (Core),3.10.0-1127.8.2.el7.x86_64


### Chassis

,model,vendor
csd3:cclake-ib-icc19-impi19-ucx,PowerEdge C6420,Dell Inc.
alaska:ib-gcc9-openmpi4-ucx,PowerEdge R630,Dell Inc.


### CPU

,architecture,model,cpus /node,sockets /node,cores /socket,threads /core
csd3:cclake-ib-icc19-impi19-ucx,x86_64,Intel(R) Xeon(R) Platinum 8276 CPU @ 2.20GHz,56,2,28,1
alaska:ib-gcc9-openmpi4-ucx,x86_64,Intel(R) Xeon(R) CPU E5-2683 v4 @ 2.10GHz,64,2,16,2


### Memory

,memory /node,type
csd3:cclake-ib-icc19-impi19-ucx,196G,Unbuffered-DDR4
alaska:ib-gcc9-openmpi4-ucx,131G,Registered-DDR4


In [5]:
IFACE_DETAILED_PARAMS = ['features', 'pause_opts', 'ring_max', 'ring_curr']

# Gather network data: this is a bit different in that sysinfo['net'] -> {iface1:{parameters..}, iface2:{parameters...}, ...}    
netvals = defaultdict(dict)
for syspart, sysinfo in sysinfos.items():
    for hostname, nodedata in sysinfo.items():
        for iface in nodedata['net']:
            for k, v in nodedata['net'][iface].items():
                pw = '%s: %s' % (syspart, iface)
                if isinstance(v, list):
                    v = ','.join(v)
                netvals[pw].setdefault(k, set()).add(v)
                
df = pd.DataFrame(netvals)
df = df.applymap(modules.utils.singleval)
df = df.transpose()

display_markdown('### Network adaptors - general information', raw=True)
general_df = df[df.columns.difference(IFACE_DETAILED_PARAMS)]
display(general_df)

display_markdown('### Network adaptors - detailed parameters', raw=True)
details_df = df[IFACE_DETAILED_PARAMS]
display(details_df)

### Network adaptors - general information

,card,descr,link_layer,port,rate,speed
csd3:cclake-ib-icc19-impi19-ucx: ib0,mlx5_0,5e:00.0 Infiniband controller [0207]: Mellanox Technologies MT28908 Family [ConnectX-6] [15b3:101b],InfiniBand,1,100 Gb/sec (2X HDR),100000 Mbits/s
csd3:cclake-ib-icc19-impi19-ucx: p4p2,mlx5_1,5e:00.1 Ethernet controller [0200]: Mellanox Technologies MT28908 Family [ConnectX-6] [15b3:101b],Ethernet,1,50 Gb/sec (2X EDR),50000 Mbits/s
alaska:ib-gcc9-openmpi4-ucx: em1,NaN,01:00.0 Ethernet controller [0200]: Intel Corporation 82599ES 10-Gigabit SFI/SFP+ Network Connection [8086:10fb] (rev 01),NaN,NaN,NaN,10000 Mbits/s
alaska:ib-gcc9-openmpi4-ucx: ib0,NaN,03:00.0 Infiniband controller [0207]: Mellanox Technologies MT27700 Family [ConnectX-4] [15b3:1013],InfiniBand,NaN,100 Gb/sec (4X EDR),100000 Mbits/s
alaska:ib-gcc9-openmpi4-ucx: p3p2,NaN,03:00.1 Ethernet controller [0200]: Mellanox Technologies MT27700 Family [ConnectX-4] [15b3:1013],Ethernet,NaN,25 Gb/sec (1X EDR),25000 Mbits/s


### Network adaptors - detailed parameters

,features,pause_opts,ring_max,ring_curr
csd3:cclake-ib-icc19-impi19-ucx: ib0,"rx-checksumming=on,tx-checksumming=on,tx-checksum-ipv4=on,tx-checksum-ip-generic=off [fixed],tx-checksum-ipv6=on,tx-checksum-fcoe-crc=off [fixed],tx-checksum-sctp=off [fixed],scatter-gather=on,tx-scatter-gather=on,tx-scatter-gather-fraglist=off [fixed],tcp-segmentation-offload=on,tx-tcp-segmentation=on,tx-tcp-ecn-segmentation=off [fixed],tx-tcp6-segmentation=on,tx-tcp-mangleid-segmentation=off,udp-fragmentation-offload=off [fixed],generic-segmentation-offload=on,generic-receive-offload=on,large-receive-offload=on,rx-vlan-offload=off [fixed],tx-vlan-offload=off [fixed],ntuple-filters=off [fixed],receive-hashing=on,highdma=on [fixed],rx-vlan-filter=off [fixed],vlan-challenged=on [fixed],tx-lockless=off [fixed],netns-local=off [fixed],tx-gso-robust=off [fixed],tx-fcoe-segmentation=off [fixed],tx-gre-segmentation=off [fixed],tx-ipip-segmentation=off [fixed],tx-sit-segmentation=off [fixed],tx-udp_tnl-segmentation=off [fixed],fcoe-mtu=off [fixed],tx-nocache-copy=off,loopback=off [fixed],rx-fcs=off [fixed],rx-all=off [fixed],tx-vlan-stag-hw-insert=off [fixed],rx-vlan-stag-hw-parse=off [fixed],rx-vlan-stag-filter=off [fixed],busy-poll=off [fixed],tx-gre-csum-segmentation=off [fixed],tx-udp_tnl-csum-segmentation=off [fixed],tx-gso-partial=off [fixed],tx-sctp-segmentation=off [fixed],rx-gro-hw=off [fixed],l2-fwd-offload=off [fixed],hw-tc-offload=off [fixed],rx-udp_tunnel-port-offload=off [fixed]",,"RX=8192,RX Mini=0,RX Jumbo=0,TX=8192","RX=512,RX Mini=0,RX Jumbo=0,TX=1024"
csd3:cclake-ib-icc19-impi19-ucx: p4p2,"rx-checksumming=on,tx-checksumming=on,tx-checksum-ipv4=off [fixed],tx-checksum-ip-generic=on,tx-checksum-ipv6=off [fixed],tx-checksum-fcoe-crc=off [fixed],tx-checksum-sctp=off [fixed],scatter-gather=on,tx-scatter-gather=on,tx-scatter-gather-fraglist=off [fixed],tcp-segmentation-offload=on,tx-tcp-segmentation=on,tx-tcp-ecn-segmentation=off [fixed],tx-tcp6-segmentation=on,tx-tcp-mangleid-segmentation=off,udp-fragmentation-offload=off [fixed],generic-segmentation-offload=on,generic-receive-offload=on,large-receive-offload=off,rx-vlan-offload=on,tx-vlan-offload=on,ntuple-filters=off,receive-hashing=on,highdma=on [fixed],rx-vlan-filter=on,vlan-challenged=off [fixed],tx-lockless=off [fixed],netns-local=off [fixed],tx-gso-robust=off [fixed],tx-fcoe-segmentation=off [fixed],tx-gre-segmentation=on,tx-ipip-segmentation=off [fixed],tx-sit-segmentation=off [fixed],tx-udp_tnl-segmentation=on,fcoe-mtu=off [fixed],tx-nocache-copy=off,loopback=off [fixed],rx-fcs=off,rx-all=off,tx-vlan-stag-hw-insert=on,rx-vlan-stag-hw-parse=off [fixed],rx-vlan-stag-filter=on [fixed],busy-poll=off [fixed],tx-gre-csum-segmentation=on,tx-udp_tnl-csum-segmentation=on,tx-gso-partial=on,tx-sctp-segmentation=off [fixed],rx-gro-hw=off [fixed],l2-fwd-offload=off [fixed],hw-tc-offload=off,rx-udp_tunnel-port-offload=on","Autonegotiate=off,RX=on,TX=on","RX=8192,RX Mini=0,RX Jumbo=0,TX=8192","RX=1024,RX Mini=0,RX Jumbo=0,TX=1024"
alaska:ib-gcc9-openmpi4-ucx: em1,"rx-checksumming=on,tx-checksumming=on,tx-checksum-ipv4=off [fixed],tx-checksum-ip-generic=on,tx-checksum-ipv6=off [fixed],tx-checksum-fcoe-crc=on [fixed],tx-checksum-sctp=on,scatter-gather=on,tx-scatter-gather=on,tx-scatter-gather-fraglist=off [fixed],tcp-segmentation-offload=on,tx-tcp-segmentation=on,tx-tcp-ecn-segmentation=off [fixed],tx-tcp6-segmentation=on,tx-tcp-mangleid-segmentation=off,udp-fragmentation-offload=off [fixed],generic-segmentation-offload=on,generic-receive-offload=on,large-receive-offload=off,rx-vlan-offload=on,tx-vlan-offload=on,ntuple-filters=off,receive-hashing=on,highdma=on [fixed],rx-vlan-filter=on,vlan-challenged=off [fixed],tx-lockless=off [fixed],netns-local=off [fixed],tx-gso-robust=off [fixed],tx-fcoe-segmentation=on [fixed],tx-gre-segmentation=on,tx-ipip-segmentation=on,tx-sit-segmentation=on,tx-udp_tnl-segmentation=on,fcoe-mtu=off [fixed],tx-nocache-copy=off,loopback=off [fixed],rx-fcs=off [fixed],rx-all=of

In [6]:
# # just to show which way around we want it:

# example = [['128GB', '24'],
#            ['256GB', '128'],
#            ['64GB', '6']
#           ]

# index = ['medium sys', 'large sys', 'small sys']
# cols = ['memory', 'num nodes']
# df = pd.DataFrame(example, index=index, columns=cols, dtype=str)
# display(df)